In [1]:
import pandas as pd
import glob
import igraph as ig
import numpy as np
import re
from tqdm import tqdm

In [7]:
# Download associations https://platform.opentargets.org/downloads
!wget --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/association_by_datasource_direct -P data/open_targets_25_09/association_type


--2025-11-26 00:22:33--  ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/association_by_datasource_direct
           => '.listing'
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/opentargets/platform/25.09/output ... done.
==> PASV ... done.    ==> LIST ... done.

.listing                [ <=>                ]   2.90K  --.-KB/s    in 0s      

2025-11-26 00:22:35 (101 MB/s) - '.listing' saved [2965]

Removed '.listing'.
--2025-11-26 00:22:35--  ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/association_by_datasource_direct/association_by_datasource_direct
           => '.listing'
==> CWD (1) /pub/databases/opentargets/platform/25.09/output/association_by_datasource_direct ... done.
==> PASV ... done.    ==> LIST ... done.

.listing   

==> PASV ... done.    ==> RETR part-00010-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet ... done.
Length: 1482138 (1.4M)

part-00010-0009005d 100%[===================>]   1.41M  5.12MB/s    in 0.3s    

2025-11-26 00:22:40 (5.12 MB/s) - 'part-00010-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet' saved [1482138]

--2025-11-26 00:22:40--  ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/association_by_datasource_direct/part-00011-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet
           => 'part-00011-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet'
==> CWD not required.
==> PASV ... done.    ==> RETR part-00011-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet ... done.
Length: 1266719 (1.2M)

part-00011-0009005d 100%[===================>]   1.21M  3.89MB/s    in 0.3s    

2025-11-26 00:22:41 (3.89 MB/s) - 'part-00011-0009005d-4472-49b8-9b32-378935bacdd2-c000.snappy.parquet' saved [1266719]

--2025-11-26 00:22:41--  ftp:/

In [8]:
data_dir = "data/open_targets_25_09/association_type/"
df = pd.read_parquet(data_dir, engine="pyarrow")
print(df.head())

         datatypeId       datasourceId    diseaseId         targetId  \
0  affected_pathway  cancer_biomarkers  EFO_0000095  ENSG00000010671   
1  affected_pathway  cancer_biomarkers  EFO_0000095  ENSG00000110848   
2  affected_pathway  cancer_biomarkers  EFO_0000095  ENSG00000165025   
3  affected_pathway  cancer_biomarkers  EFO_0000095  ENSG00000197943   
4  affected_pathway  cancer_biomarkers  EFO_0000178  ENSG00000066468   

      score  evidenceCount  
0  0.607931              1  
1  0.607931              1  
2  0.607931              1  
3  0.607931              1  
4  0.607931              1  


In [9]:
# Remove influence of literature
df = df[df['datatypeId'] != 'literature']
df['datatypeId'].value_counts()

datatypeId
genetic_association    714130
animal_model           690354
rna_expression         166503
somatic_mutation        96504
known_drug              74187
affected_pathway        38121
Name: count, dtype: int64

In [10]:
df

,datatypeId,datasourceId,diseaseId,targetId,score,evidenceCount
0,affected_pathway,cancer_biomarkers,EFO_0000095,ENSG00000010671,0.607931,1
1,affected_pathway,cancer_biomarkers,EFO_0000095,ENSG00000110848,0.607931,1
2,affected_pathway,cancer_biomarkers,EFO_0000095,ENSG00000165025,0.607931,1
3,affected_pathway,cancer_biomarkers,EFO_0000095,ENSG00000197943,0.607931,1
4,affected_pathway,cancer_biomarkers,EFO_0000178,ENSG00000066468,0.607931,1
...,...,...,...,...,...,...
4200230,genetic_association,clingen,Orphanet_93271,ENSG00000118965,0.607931,1
4200231,genetic_association,clingen,Orphanet_93271,ENSG00000119333,0.607931,1
4200232,genetic_association,clingen,Orphanet_93271,ENSG00000126870,0.607931,1
4200233,genetic_association,clingen,Orphanet_98673,ENSG00000100412,0.607931,1


In [11]:
weights = {
    "europe_pm": 0.2,
    "expression_atlas": 0.2,
    "impc": 0.2,
    "progeny": 0.5,
    "slapenrich": 0.5,
    "cancer_biomarkers": 0.5,
    "sysbio": 0.5,
    "otar_projects": 0.5,
    "default": 1.0  
}

def get_weight(ds):
    return weights.get(ds, weights['default'])
df['weight'] = df['datatypeId'].apply(get_weight)
df['weighted_term'] = 1 - (df['weight'] * df['score'])

def harmonic_sum(group):
    product_term = np.prod(group['weighted_term'])
    return 1 - product_term
final_scores = df.groupby(['diseaseId', 'targetId']).apply(harmonic_sum).reset_index(name='customAssociationScore')
final_scores

/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_54942/1002392638.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_scores = df.groupby(['diseaseId', 'targetId']).apply(harmonic_sum).reset_index(name='customAssociationScore')


,diseaseId,targetId,customAssociationScore
0,DOID_10113,ENSG00000011677,0.060793
1,DOID_10113,ENSG00000022355,0.060793
2,DOID_10113,ENSG00000100342,0.607931
3,DOID_10113,ENSG00000113327,0.060793
4,DOID_10113,ENSG00000113578,0.531939
...,...,...,...
1725155,Orphanet_99947,ENSG00000198836,0.281654
1725156,Orphanet_99947,ENSG00000198862,0.290713
1725157,Orphanet_99947,ENSG00000250120,0.401189
1725158,Orphanet_99947,ENSG00000264424,0.252291


In [13]:
# Download from OpenTargets
# !wget --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/evidence -P data/open_targets_25_09
# !wget --recursive --no-parent --no-host-directories --cut-dirs 8 ftp://ftp.ebi.ac.uk/pub/databases/opentargets/platform/25.09/output/disease -P data/open_targets_25_09

# file_list = glob.glob("data/open_targets_25_09/targets/*.json")
# data_frames_list = [pd.read_json(file,lines="TRUE") for file in file_list]
# combined_targets = pd.concat(data_frames_list, ignore_index=True)
# combined_targets = combined_targets.rename(columns={'id': 'targetId'})
# # df = pd.read_parquet(data_dir, engine="pyarrow")
# combined_targets.to_csv("data/open_targets_25_09/gene_map.csv", index=False)

# file_list = glob.glob("data/open_targets_25_09/diseases/*.json")
# data_frames_list = [pd.read_json(file,lines="TRUE") for file in file_list]
# combined_disease = pd.concat(data_frames_list, ignore_index=True)
# combined_disease = combined_disease.rename(columns={'id': 'diseaseId'})
# # df = pd.read_parquet(data_dir, engine="pyarrow")
# combined_disease.to_csv("data/open_targets_25_09/disease_map.csv", index=False)

combined_disease = pd.read_csv("data/open_targets_25_09/disease_map.csv")
combined_disease.columns
combined_disease = combined_disease[["diseaseId","name"]]

combined_targets = pd.read_csv("data/open_targets_25_09/gene_map.csv")
combined_targets.columns
combined_targets = combined_targets[["targetId","approvedSymbol"]]

In [14]:
display(combined_disease.head(5))
display(combined_targets.head(5))

,diseaseId,name
0,EFO_0001073,obesity
1,EFO_0003105,spina bifida
2,EFO_0003812,refractory anemia with ringed sideroblasts
3,EFO_0004238,hearing loss
4,EFO_0004586,complete blood cell count


,targetId,approvedSymbol
0,ENSG00000007202,BLTP2
1,ENSG00000011260,UTP18
2,ENSG00000064489,BORCS8-MEF2B
3,ENSG00000064655,EYA2
4,ENSG00000064703,DDX20


In [15]:
disease_edge = pd.merge(final_scores, combined_disease, on = "diseaseId")
disease_edge = pd.merge(disease_edge, combined_targets, on = 'targetId')
disease_edge = disease_edge[["name","approvedSymbol","customAssociationScore"]]
# disease_edge.to_csv("gene_association_edge_list_no_lit.csv.gzip", compression="gzip", index=False)
# disease_edge = pd.read_csv("gene_association_edge_list_no_lit.csv")
disease_edge

,name,approvedSymbol,customAssociationScore
0,trypanosomiasis,GABRA3,0.060793
1,trypanosomiasis,GABRA1,0.060793
2,trypanosomiasis,APOL1,0.607931
3,trypanosomiasis,GABRG2,0.060793
4,trypanosomiasis,FGF1,0.531939
...,...,...,...
1580720,Autosomal dominant Charcot-Marie-Tooth disease...,OPA1,0.281654
1580721,Autosomal dominant Charcot-Marie-Tooth disease...,LTN1,0.290713
1580722,Autosomal dominant Charcot-Marie-Tooth disease...,PCDHA10,0.401189
1580723,Autosomal dominant Charcot-Marie-Tooth disease...,MYH4,0.252291


In [16]:
measurement_keywords = ['measurement', 'level', 'concentration', 'trait',
                        'response', 'ratio', 'presence', 'status', 'microdeletion', 
                        'microduplication', 'deletion', 'duplication']
    
# Assuming disease_edge is your DataFrame
pattern = '|'.join(measurement_keywords)  # Create regex pattern

# Filter out rows where 'name' contains any of the keywords (case-insensitive)
disease_edge_filtered = disease_edge[~disease_edge['name'].str.contains(pattern, case=False, na=False)]
disease_edge_filtered

,name,approvedSymbol,customAssociationScore
0,trypanosomiasis,GABRA3,0.060793
1,trypanosomiasis,GABRA1,0.060793
2,trypanosomiasis,APOL1,0.607931
3,trypanosomiasis,GABRG2,0.060793
4,trypanosomiasis,FGF1,0.531939
...,...,...,...
1580720,Autosomal dominant Charcot-Marie-Tooth disease...,OPA1,0.281654
1580721,Autosomal dominant Charcot-Marie-Tooth disease...,LTN1,0.290713
1580722,Autosomal dominant Charcot-Marie-Tooth disease...,PCDHA10,0.401189
1580723,Autosomal dominant Charcot-Marie-Tooth disease...,MYH4,0.252291


In [17]:
disease_edge_target = disease_edge_filtered.pivot_table(index = "approvedSymbol",columns = "name",values = "customAssociationScore",aggfunc = "mean")
disease_edge_target = disease_edge_target.fillna(0)
disease_edge_target

name,15q overgrowth syndrome,"17,20-lyase deficiency, isolated","17-alpha-hydroxylase/17,20-lyase deficiency, combined complete","17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",2-3 finger syndactyly,2-3 toe syndactyly,2-aminoadipic 2-oxoadipic aciduria,2-hydroxyglutaric aciduria,2-methylbutyryl-CoA dehydrogenase deficiency,3-4 toe syndactyly,...,xeroderma pigmentosum group G,xeroderma pigmentosum variant type,xeroderma pigmentosum-Cockayne syndrome complex,xerosis and growth failure with immune and pulmonary dysfunction syndrome,xerostomia,young-onset Parkinson disease,zebra body myopathy,"zinc deficiency, transient neonatal",zygodactyly type 3,Åland Islands eye disease
approvedSymbol,,,,,,,,,,,,,,,,,,,,,
7SK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Transpose to get diseases as rows and genes as columns
disease_matrix = disease_edge_target.T  # shape: (num_diseases, num_genes)
X = disease_matrix.values
disease_names = disease_matrix.index.to_list()
n_diseases = len(disease_names)

# Initialize binary similarity matrix
dis_dis_matrix = np.zeros((n_diseases, n_diseases), dtype=float)

# Only compute upper triangle to avoid redundancy
for i in tqdm(range(n_diseases), desc="Computing shared gene matrix"):
    xi = X[i] > 0  # binary presence vector
    for j in range(i, n_diseases):
        if i == j:
            value = 0
        else:
            shared_arr = np.minimum(X[i], X[j])
            value = shared_arr[shared_arr != 0].mean()  # mean of non-zero elements (average association)
        dis_dis_matrix[i, j] = value
        dis_dis_matrix[j, i] = value  # symmetry

# Convert to DataFrame
dis_dis_df = pd.DataFrame(dis_dis_matrix, index=disease_names, columns=disease_names)
np.fill_diagonal(dis_dis_df.values, 0)
# Save or preview
# binary_df.to_csv("disease_disease_shared_gene_binary.csv")
dis_dis_df = dis_dis_df.fillna(0)
dis_dis_df.head(3)


Computing shared gene matrix:   0%|                                                                                                                                                                                        | 0/13371 [00:00<?, ?it/s]/var/folders/2h/45bqs84942sfxfwfndsxtgs40000gn/T/ipykernel_54942/1760117136.py:18: RuntimeWarning: Mean of empty slice.
  value = shared_arr[shared_arr != 0].mean()  # mean of non-zero elements (average association)
/Users/vladimirk/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Computing shared gene matrix: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13371/13371 [41:07<00:00,  5.42it/s]


,15q overgrowth syndrome,"17,20-lyase deficiency, isolated","17-alpha-hydroxylase/17,20-lyase deficiency, combined complete","17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",2-3 finger syndactyly,2-3 toe syndactyly,2-aminoadipic 2-oxoadipic aciduria,2-hydroxyglutaric aciduria,2-methylbutyryl-CoA dehydrogenase deficiency,3-4 toe syndactyly,...,xeroderma pigmentosum group G,xeroderma pigmentosum variant type,xeroderma pigmentosum-Cockayne syndrome complex,xerosis and growth failure with immune and pulmonary dysfunction syndrome,xerostomia,young-onset Parkinson disease,zebra body myopathy,"zinc deficiency, transient neonatal",zygodactyly type 3,Åland Islands eye disease
15q overgrowth syndrome,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.281715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"17,20-lyase deficiency, isolated",0.0,0.000000,0.871953,0.847936,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"17-alpha-hydroxylase/17,20-lyase deficiency, combined complete",0.0,0.871953,0.000000,0.847936,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# Edge between diseases in disaese-disease matrix is calculated as Pearson correlation between their gene vectors
# data = disease_edge_target.to_numpy()
# correlations = np.corrcoef(data, rowvar=False)  # rowvar=False means columns are variables

In [16]:
#  Bray-Curtis dissimilarity (e.g., sum of min over shared genes)
# More computationally expensive, but more biologically interpretable
# Bray, J. R., & Curtis, J. T. (1957). An ordination of the upland forest communities of southern Wisconsin. Ecological Monographs, 27(4), 325–349.
# Lozupone, C. A., & Knight, R. (2005). UniFrac: a new phylogenetic method for comparing microbial communities. Applied and Environmental Microbiology, 71(12), 8228–8235.

# diseases = disease_edge_target.columns
# adj_matrix_min = pd.DataFrame(0.0, index=diseases, columns=diseases)

# for i in tqdm(range(len(diseases)), desc="Computing min adjacency"):
#     for j in range(i, len(diseases)):
#         d1, d2 = diseases[i], diseases[j]
#         shared_min = np.minimum(disease_edge_target[d1], disease_edge_target[d2]).sum()
#         adj_matrix_min.loc[d1, d2] = shared_min
#         adj_matrix_min.loc[d2, d1] = shared_min  # symmetric


In [50]:
# correlations = pd.DataFrame(
#     correlations,
#     index=disease_edge_target.columns,
#     columns=disease_edge_target.columns
# )
# correlations.head()

In [47]:
# cor_df = correlations.apply(pd.to_numeric)
# np.fill_diagonal(cor_df.values, 0)
# cor_df[cor_df < 0] = 0
# x = np.quantile(np.abs(cor_df.values.flatten()), 0.95)
# cor_df[cor_df < x] = 0
# cor_df.head()

In [19]:
dis_dis_df

,15q overgrowth syndrome,"17,20-lyase deficiency, isolated","17-alpha-hydroxylase/17,20-lyase deficiency, combined complete","17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",2-3 finger syndactyly,2-3 toe syndactyly,2-aminoadipic 2-oxoadipic aciduria,2-hydroxyglutaric aciduria,2-methylbutyryl-CoA dehydrogenase deficiency,3-4 toe syndactyly,...,xeroderma pigmentosum group G,xeroderma pigmentosum variant type,xeroderma pigmentosum-Cockayne syndrome complex,xerosis and growth failure with immune and pulmonary dysfunction syndrome,xerostomia,young-onset Parkinson disease,zebra body myopathy,"zinc deficiency, transient neonatal",zygodactyly type 3,Åland Islands eye disease
15q overgrowth syndrome,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.281715,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17,20-lyase deficiency, isolated",0.0,0.000000,0.871953,0.847936,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17-alpha-hydroxylase/17,20-lyase deficiency, combined complete",0.0,0.871953,0.000000,0.847936,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",0.0,0.847936,0.847936,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2-3 finger syndactyly,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.252291,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young-onset Parkinson disease,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.258036,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
zebra body myopathy,0.0,0.000000,0.000000,0.000000,0.252291,0.0,0.0,0.0,0.0,0.0,...,0.285484,0.0,0.0,0.0,0.0,0.0,0.000000,0.274724,0.0,0.0
"zinc deficiency, transient neonatal",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.274724,0.000000,0.0,0.0
zygodactyly type 3,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [20]:
# Remove rows with all zeros
dis_dis_df = dis_dis_df.loc[~(dis_dis_df == 0).all(axis=1)]

# Remove columns with all zeros
dis_dis_df = dis_dis_df.loc[:, ~(dis_dis_df == 0).all(axis=0)]
dis_dis_df

,15q overgrowth syndrome,"17,20-lyase deficiency, isolated","17-alpha-hydroxylase/17,20-lyase deficiency, combined complete","17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",2-3 finger syndactyly,2-3 toe syndactyly,2-aminoadipic 2-oxoadipic aciduria,2-hydroxyglutaric aciduria,2-methylbutyryl-CoA dehydrogenase deficiency,3-4 toe syndactyly,...,xeroderma pigmentosum group G,xeroderma pigmentosum variant type,xeroderma pigmentosum-Cockayne syndrome complex,xerosis and growth failure with immune and pulmonary dysfunction syndrome,xerostomia,young-onset Parkinson disease,zebra body myopathy,"zinc deficiency, transient neonatal",zygodactyly type 3,Åland Islands eye disease
15q overgrowth syndrome,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.281715,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17,20-lyase deficiency, isolated",0.0,0.000000,0.871953,0.847936,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17-alpha-hydroxylase/17,20-lyase deficiency, combined complete",0.0,0.871953,0.000000,0.847936,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
"17-alpha-hydroxylase/17,20-lyase deficiency, combined partial",0.0,0.847936,0.847936,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2-3 finger syndactyly,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.252291,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young-onset Parkinson disease,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.258036,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
zebra body myopathy,0.0,0.000000,0.000000,0.000000,0.252291,0.0,0.0,0.0,0.0,0.0,...,0.285484,0.0,0.0,0.0,0.0,0.0,0.000000,0.274724,0.0,0.0
"zinc deficiency, transient neonatal",0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.274724,0.000000,0.0,0.0
zygodactyly type 3,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


In [21]:
all_values = dis_dis_df.values.flatten()

# Count zeros and non-zeros
total_values = len(all_values)
zero_count = np.count_nonzero(all_values == 0)
non_zero_values = all_values[all_values != 0]

# Calculate percentage of zeros
zero_percentage = (zero_count / total_values) * 100
print(f"Percentage of zeros: {zero_percentage:.2f}%")

Percentage of zeros: 79.30%


In [23]:
dis_dis_df.to_csv("data/open_targets_25_09/dis_dis_nolit_25_09.csv", index=False)

In [24]:
import secrets
print(secrets.token_urlsafe(32))

T38570ezxcMH9Elaio4TU3GxW8vNI2Fsatnyd5QSeT8


In [ ]:
for col in cor_df.columns:
    if 'lzh' in col:
        print(col)

In [63]:
# Generate disease modules
def create_disease_module(df, disease_name, output_file):
    disease_col = df.loc[:, disease_name]
    top_20 = (
        disease_col.sort_values(ascending=False)
        .head(20)
    )
    display(top_20)
    pd.DataFrame(top_20.index).to_csv(output_file, index=False)

create_disease_module(cor_df, "Alzheimer disease", "../data/open_targets/ad_cor_20_no_lit.csv")
create_disease_module(cor_df, "childhood cancer", "../data/open_targets/childhoodcancer_cor_20_no_lit.csv")
create_disease_module(cor_df, "congenital heart disease", "../data/open_targets/chd_cor_20_no_lit.csv")

name
Parkinson disease                        0.739253
lysosomal storage disease                0.709381
multiple sclerosis                       0.687098
polygenic risk score                     0.318300
neurodegenerative disease                0.289255
Dementia                                 0.247351
family history of Alzheimer’s disease    0.232589
neuroinflammatory disorder               0.229831
schizoaffective disorder                 0.197127
late-onset Alzheimers disease            0.184979
schizophrenia                            0.181444
Cognitive impairment                     0.169687
depressive disorder                      0.169549
dementia                                 0.162336
delirium                                 0.160479
post-operative sign or symptom           0.154135
pain                                     0.152318
bipolar disorder                         0.150946
obesity                                  0.147595
treatment resistant depression           0.14

name
major salivary gland cancer                                    0.939302
small intestine lymphoma                                       0.614396
Blast Phase Chronic Myelogenous Leukemia, BCR-ABL1 Positive    0.602049
core binding factor acute myeloid leukemia                     0.508395
nephrogenic fibrosing dermopathy                               0.484955
central nervous system leukemia                                0.481484
lymphoid leukemia                                              0.456604
adult glioblastoma                                             0.448029
refractory hairy cell leukemia                                 0.408844
diffuse intrinsic pontine glioma                               0.398142
loiasis                                                        0.378596
chronic myelogenous leukemia                                   0.340391
fibromatosis                                                   0.329704
Pineoblastoma                                              

name
Abnormal heart morphology                                       0.499012
scimitar syndrome                                               0.484181
right atrial isomerism                                          0.460492
visceral heterotaxy                                             0.450829
congenital heart defects, multiple types, 6                     0.446368
tricuspid atresia                                               0.429050
Heterotaxia                                                     0.428191
heterotaxy, visceral, 12, autosomal                             0.423864
hypoplastic left heart syndrome                                 0.423560
atrial septal defect, ostium primum type                        0.423502
atrial septal defect 1                                          0.423022
persistent truncus arteriosus                                   0.420995
Truncus arteriosus                                              0.420755
Congenitally uncorrected transposition of the 

In [ ]:
# Run CTD-ID
# python Python/main_gba.py --adj_path data/open_targets/cor_matrix_95_nolit.csv --s_nodes data/open_targets/ad_cor_20_no_lit.csv -o data/open_targets/results
# python Python/main_gba.py --adj_path data/open_targets/cor_matrix_95_nolit.csv --s_nodes data/open_targets/childhoodcancer_cor_20_no_lit.csv -o data/open_targets/results
# python Python/main_gba.py --adj_path data/open_targets/cor_matrix_95_nolit.csv --s_nodes data/open_targets/chd_cor_20_no_lit.csv -o data/open_targets/results